# Imports and reading data

In [ ]:
import numpy as np
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime

MortalityFilename = 'mortality/Mortality_AGBLSterbewoche.csv'
MortalityData = pd.read_csv(MortalityFilename, delimiter = ";",header=0)

CrawlerFilename = 'mortality/DataMerged.csv'
CrawlerData = pd.read_csv(CrawlerFilename, delimiter = ",", names= ["Deathdate","Name","Age"])

# Normalize Numbers by Inhabitants of State

In [ ]:
Berlin = MortalityData[MortalityData['Bundesland']=='Berlin']
MortalityData.sort_values(by='Bundesland', inplace=True)

States = list(set(MortalityData['Bundesland']))
States.sort()
Inhabitants = pd.Series([10952,12931,3575,2495,679,1810,6213,1611,7946,17890,4066,997,4082,2236,2882,2158], index=States)
print(Inhabitants)


In [ ]:
# Normalize the data with respect to inhabitants of the state

NormalMort = MortalityData.copy()

for i, State in enumerate(States):
    NormalMort.loc[NormalMort.Bundesland==State, 'Todesfälle'] = MortalityData.loc[MortalityData.Bundesland==State, 'Todesfälle'].div(Inhabitants[State])



# Mortality rates over the year for each state

In [ ]:
#AgeGroups = set(MortalityData['Altersgruppe'])
AgeGroups = ['0-64', '65-74', '75-84', '85+']

#States = ['Hamburg', 'Bremen', 'Berlin']

Colors = ['g', 'c', 'b', 'r']

Fig, Axes = plt.subplots(4, 4, sharex=True, figsize=(15,15))
Fig.tight_layout()
StateIndex = 0
Weeks = range(53)
MeanPerWeek = np.zeros((16, 4, 53))
SumPerWeek = np.zeros((16, 4, 53))
MedianPerWeek = np.zeros((16, 4, 53))

MeanPerWeekN = np.zeros((16, 4, 53))
SumPerWeekN = np.zeros((16, 4, 53))
MedianPerWeekN = np.zeros((16, 4, 53))


for State in States:
    for Num, AgeGroup in enumerate(AgeGroups):
        FilteredCases = MortalityData[MortalityData['Bundesland']==State]
        FilteredCases = FilteredCases[FilteredCases['Altersgruppe']==AgeGroup] # only contains Cases for State and AgeGroup
        FilteredCasesN = NormalMort[NormalMort['Bundesland']==State]
        FilteredCasesN = FilteredCasesN[FilteredCasesN['Altersgruppe']==AgeGroup] 
        for i in range(1,54):
            MeanPerWeek[StateIndex, Num, i-1] = FilteredCases[FilteredCases['Sterbewoche']==i]['Todesfälle'].mean()
            SumPerWeek[StateIndex, Num, i-1] = FilteredCases[FilteredCases['Sterbewoche']==i]['Todesfälle'].sum()
            MedianPerWeek[StateIndex, Num, i-1] = FilteredCases[FilteredCases['Sterbewoche']==i]['Todesfälle'].median()
        
            MeanPerWeekN[StateIndex, Num, i-1] = FilteredCasesN[FilteredCases['Sterbewoche']==i]['Todesfälle'].mean()
            SumPerWeekN[StateIndex, Num, i-1] = FilteredCasesN[FilteredCases['Sterbewoche']==i]['Todesfälle'].sum()
            MedianPerWeekN[StateIndex, Num, i-1] = FilteredCasesN[FilteredCases['Sterbewoche']==i]['Todesfälle'].median()
        
        Axes[StateIndex%4,StateIndex//4].plot(Weeks, MeanPerWeek[StateIndex, Num, :], Colors[Num])
        Linestyle = Colors[Num] + '--'
        Kurz = FilteredCases[FilteredCases.Sterbejahr==2004]['Todesfälle']
        Axes[StateIndex%4,StateIndex//4].plot(Weeks, Kurz, Linestyle)
        
        Axes[StateIndex%4, StateIndex//4].set_title(State)
        # plot numbers: oben-links, columns runter, dann nach rechts weiter
        
    StateIndex += 1
   
plt.show()

# Germany as a whole: How many people are dying at what age?

In [ ]:
# wie viele Leute sterben pro Altersgruppe?
Summiert = np.sum(np.sum(SumPerWeek, axis=0), axis=1)
plt.figure()
plt.title("Ganz Deutschland: Summe pro Altersgruppe")
plt.bar([0,1,2,3],Summiert)
plt.xticks([0.5,1.5,2.5,3.5], ['0-64', '65-74', '75-84', '85+'])
plt.show()

# Age distribution of people dying?

In [ ]:
# Plot: linie für jede Altersgruppe, x-achse: jahre, y-achse: gesamt-fälle
# (State, Age, Week)
MortalityData.loc[MortalityData.Bundesland==State, 'Todesfälle'].div(Inhabitants[State])

MeanPerYear = np.zeros((4,15))
#MeanNormalizedYear = np.zeros((4,15))

Fig, Ax = plt.subplots(1,1,figsize=(9,5))
plt.suptitle("Pro Altersgruppe: Fälle über die Jahre")

for Num, AgeGroup in enumerate(AgeGroups):
    FilteredCases = MortalityData[MortalityData['Altersgruppe']==AgeGroup]# only contains Cases for State and AgeGroup
    FilteredNormal = NormalMort[NormalMort['Altersgruppe']==AgeGroup]
    i = 0
    for Jahr in range(2001, 2016):
        MeanPerYear[Num, i] = FilteredCases[FilteredCases['Sterbejahr']==Jahr]['Todesfälle'].mean()
        #MeanNormalizedYear[Num, i] = FilteredNormal[FilteredNormal['Sterbejahr']==Jahr]['Todesfälle'].mean()
        i += 1
    x = range(15)
    Ax.plot(x, MeanPerYear[Num, :], Colors[Num])
    #Ax[1].plot(range(15), MeanNormalizedYear[Num, :], Colors[Num])
    plt.legend(['0-64', '65-74', '75-84', '85+'], loc=2)
    Ax.set_xticks(x)#, ['1', '2', '3','1', '2', '3','1', '2', '3','1', '2', '3'])
    xtickNames = Ax.set_xticklabels(np.arange(2001, 2016))
    
plt.show()   


# For each state

In [ ]:
# und für jedes Bundesland aufgeschlüsselt:
LandSummiert = np.sum(SumPerWeekN, axis=2)
x = np.arange(16)

Fig, Ax = plt.subplots(1,1,figsize=(9,5))
plt.suptitle("Pro Bundesland: Vergleich der Altersgruppen")
width = 0.2
Ax.bar(x-width, LandSummiert[:,0], width, color='g')
Ax.bar(x, LandSummiert[:,1], width, color='c')
Ax.bar(x+width, LandSummiert[:,2], width, color='b')
Ax.bar(x+2*width, LandSummiert[:,3], width, color='r')

Ax.set_xlim([-1,17])
ShortNames = []
for s in range(len(States)):
    ShortNames.append(States[s][:3])

print(States[8])
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
Ax.set_xticks(x, ShortNames)
xtickNames = Ax.set_xticklabels(ShortNames)
plt.setp(xtickNames, rotation=45, fontsize=9)
plt.legend(['0-64', '65-74', '75-84', '85+'], loc=4)

plt.show()


# In comparison: Berlin data

In [ ]:
NotZero = CrawlerData[CrawlerData['Age'] > -1]
Over64 = NotZero[NotZero['Age'] > 64]
Over74 = Over64[Over64['Age'] > 74]
Over84 = Over74[Over74['Age'] > 84]
Age0 = len(NotZero) - len(Over64)
Age1 = len(Over64) - len(Over74)
Age2 = len(Over74) - len(Over84)
Age3 = len(Over84)

Fig, Axes = plt.subplots(2, 3, sharex=True)
Fig.tight_layout()
Axes[0,0].set_title("Todesanzeigen")
Axes[0,0].bar([0,1,2,3],[Age0, Age1, Age2, Age3])

Axes[0,1].bar([0,1,2,3],Summiert)
Axes[0,1].set_title("Ganz D")

Anteil = [Age0, Age1, Age2, Age3] / Summiert
Axes[1,1].bar([0,1,2,3], Anteil)
Axes[1,1].set_title("TA Berlin / Fälle ganz D")

BerlinSum = np.sum(SumPerWeek[2,:,:], axis=1) # Achtung: Berlin index richtig?
Axes[0,2].bar([0,1,2,3], BerlinSum)
Axes[0,2].set_title("Offiziell Berlin")

Axes[1,2].bar([0,1,2,3], [Age0, Age1, Age2, Age3]/BerlinSum)
Axes[1,2].set_title("Anteil Berlin")


plt.xticks([0.5,1.5,2.5,3.5], ['0-64', '65-74', '75-84', '85+'])
plt.xticks(rotation=70)

plt.show()




# Crawler sample analysis
Investigating 
- distribution of age in sample
- connection of age and gender in sample
- sample coverage
- seasonal fluctuation in sample

In [ ]:
### Loading crawl data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import codecs
from dateutil.relativedelta import relativedelta

# read our file
CrawlerFile = 'mortality/Noz_death_data.csv'

def ReadCSV(filename, delimiter=','):
    data = []
    with codecs.open(filename) as f:#, encoding='ISO-8859-1') as f:
        for line in f:
            data.append(line.split(','))
    data = np.asarray(data)
    return data[1:,:] # Achtung: schmeißt die erste Zeile weg (Header)

CD = ReadCSV(CrawlerFile, delimiter=',')
# RKI: Bundesland, Sterbejahr, Sterbewoche, Altersgruppe, Todesfälle

BirthDate = []
DeathDate = []
Week = []
Age = []
DY = []
Gender = []


for i in range(CD.shape[0]): 
    #print(CD[i,:])
    DDRaw = str(CD[i,2])
    BDRaw = str(CD[i,1])
    
    DD = dt.date(int(DDRaw[-4:]),int(DDRaw[2:4]),int(DDRaw[:2]))
    BD = dt.date(int(BDRaw[-4:]),int(BDRaw[2:4]),int(BDRaw[:2]))
    BirthDate.append(BD)
    DeathDate.append(DD)
    Week.append(DD.isocalendar()[1])
    Age.append((DD-BD).total_seconds()//31557600)
    DY.append(DD.year)
    Gender.append(int(CD[i,-1]))



In [ ]:
# Construct dataframe
df = pd.DataFrame({'Age': Age,
                   'Sterbewoche': Week,
                   'Ort': CD[:,3]})

# Spalte Altersgruppe
df['Altersgruppe'] = 0
df.loc[df.Age < 65, 'Altersgruppe'] = '0-64'
df.loc[((df['Age'] >= 65 )& (df['Age'] < 75)), 'Altersgruppe'] = '65-74'
df.loc[((df['Age']  >= 75)& (df['Age'] < 85)), 'Altersgruppe'] = '75-84'
df.loc[df.Age > 84, 'Altersgruppe'] = '85+'

# Spalte Sterbejahr
df['Sterbejahr'] = DY

# Spalte Sterbedatum (sanity check)
df['DD'] = DD


## Distribution of age

In [ ]:
plt.figure()
plt.title('Distribution of Age')
plt.hist(df.Age, bins=np.linspace(0,100,50))
plt.show()


## Distribution of age for different genders

In [ ]:
# Construct dataframe
df_2 = pd.DataFrame({'Age': Age,
                   'Sterbewoche': Week,
                   'Ort': CD[:,3],
                   'Geschlecht': Gender})

df_w = df_2.loc[df_2['Geschlecht']==0]
# Spalte Altersgruppe
df_w['Altersgruppe'] = 0
df_w.loc[df_w.Age < 65, 'Altersgruppe'] = '0-64'
df_w.loc[((df_w['Age'] >= 65 )& (df['Age'] < 75)), 'Altersgruppe'] = '65-74'
df_w.loc[((df_w['Age']  >= 75)& (df['Age'] < 85)), 'Altersgruppe'] = '75-84'
df_w.loc[df_w.Age > 84, 'Altersgruppe'] = '85+'


df_m = df_2.loc[df_2['Geschlecht']==1]
# Spalte Altersgruppe
df_m['Altersgruppe'] = 0
df_m.loc[df_m.Age < 65, 'Altersgruppe'] = '0-64'
df_m.loc[((df_m['Age'] >= 65 )& (df['Age'] < 75)), 'Altersgruppe'] = '65-74'
df_m.loc[((df_m['Age']  >= 75)& (df['Age'] < 85)), 'Altersgruppe'] = '75-84'
df_m.loc[df_m.Age > 84, 'Altersgruppe'] = '85+'

# Plot
fig = plt.figure(figsize=(15, 10), dpi=80, facecolor='w', edgecolor='k')

plt.hist(df_w.Age, bins=np.linspace(0,100,50),alpha = 0.5)
plt.title("Distribution of Age Bremen: Women vs. Men", fontsize=20)
#Ax[0].set_xticklabels( ['','0-64', '65-74', '75-84', '85+'])

#NISum = np.sum(SumPerWeek[8,:,:], axis=1) # Achtung: Niedersachsen index richtig?
plt.hist(df_m.Age, bins=np.linspace(0,100,50),alpha = 0.6)
plt.xlabel("Age", fontsize=20)
plt.ylabel("Accumulated number of deaths over years", fontsize=20)
plt.legend(['women', 'men'], fontsize = 20)
plt.show()

fig.savefig("DistributionMenWoman.png")

## Distribution of age for the different age groups

In [ ]:
# und mit den Altersgruppen:
Age0 = len(df.loc[df.Altersgruppe == '0-64'])
Age1 = len(df.loc[df.Altersgruppe == '65-74'])
Age2 = len(df.loc[df.Altersgruppe == '75-84'])
Age3 = len(df.loc[df.Altersgruppe == '85+'])

Fig, Ax = plt.subplots(1,3)
Fig.tight_layout()
plt.title("Distribution of Age")
Ax[0].bar([0,1,2,3], [Age0, Age1, Age2, Age3])
Ax[0].set_title("Traueranzeigen \n Osnabrück")
#plt.xticks([0.35,1.35,2.35,3.35], ['0-64', '65-74', '75-84', '85+'])
Ax[0].set_xticklabels( ['0-64', '65-74', '75-84', '85+'])


NISum = np.sum(SumPerWeek[8,:,:], axis=1) # Achtung: Niedersachsen index richtig?
Ax[1].bar([0,1,2,3], NISum)
Ax[1].set_title("Offiziell \n Niedersachsen")
Ax[1].set_xticklabels( ['0-64', '65-74', '75-84', '85+'])

Ax[2].bar([0,1,2,3], [Age0, Age1, Age2, Age3]/NISum)
Ax[2].set_title("Anteil \n Niedersachsen")
Ax[2].set_xticklabels([['0-64', '65-74', '75-84', '85+']])

plt.show()

## Seasonal trends

In [ ]:
# Fluktuationen übers Jahr über die Altersgruppen:
Fig, Axes = plt.subplots(1,2, figsize=(9,5))
Fig.tight_layout()
StateIndex = 0
#Weeks = range(53)
Weeks = range(1,52)
SumPerWeekOsna = np.zeros((4, 53))
AgeGroups = ['0-64', '65-74', '75-84', '85+']

for Num, AgeGroup in enumerate(AgeGroups):
    FilteredCases = df[df['Altersgruppe']==AgeGroup] # only contains Cases for State and AgeGroup
    
    for i in range(1,54):
        #print(FilteredCases[FilteredCases['Sterbewoche']==i].shape[0])
        SumPerWeekOsna[Num, i-1] = FilteredCases[FilteredCases['Sterbewoche']==i].shape[0]
        
    Axes[1].plot(Weeks, SumPerWeekOsna[Num, 1:-1], Colors[Num])
    Axes[1].set_title('Traueranzeigen Osnabrück')
    Axes[0].plot(Weeks, MeanPerWeek[8, Num, 1:-1], Colors[Num])
    Axes[0].set_title('Mittelwert Niedersachsen')
    Axes[1].set_xlabel('Kalenderwoche')
    Axes[0].set_xlabel('Kalenderwoche')

    lec = plt.legend(['0-64', '65-74', '75-84', '85+'], loc=1)
    for line in lec.get_lines():
        line.set_linewidth(3.0)
   
plt.show()
Fig.savefig('SeasonalTrends.png')

## Regional coverage 

In [ ]:
# Bremens gesamtzahlen für 2015
Bremen = MortalityData[MortalityData['Bundesland']=='Niedersachsen']
Bremen2015 = Bremen[Bremen.Sterbejahr==2015]
#print(set(Bremen2015.Sterbewoche))
SumPerWeekBremen = np.zeros((4,53))
for Num, AgeGroup in enumerate(AgeGroups):
    Filtered = Bremen2015[Bremen2015.Altersgruppe==AgeGroup]
    for i in range(1,54):
        SumPerWeekBremen[Num, i-1] = Filtered[Filtered.Sterbewoche==i]['Todesfälle'].sum()

Year2015 = df.loc[df.Sterbejahr==2015]
Age0 = len(Year2015.loc[Year2015.Altersgruppe == '0-64'])
Age1 = len(Year2015.loc[Year2015.Altersgruppe == '65-74'])
Age2 = len(Year2015.loc[Year2015.Altersgruppe == '75-84'])
Age3 = len(Year2015.loc[Year2015.Altersgruppe == '85+'])

#Age0 = len(df.loc[df.Altersgruppe == '0-64'])
#Age1 = len(df.loc[df.Altersgruppe == '65-74'])
#Age2 = len(df.loc[df.Altersgruppe == '75-84'])
#Age3 = len(df.loc[df.Altersgruppe == '85+'])

AllBremen =  Age0 + Age1 + Age2 + Age3
print('Sum Bremen Orbituary Notes 2015', AllBremen)
AllBremen = 1


Fig, Ax = plt.subplots(1,3, figsize=(9,5))
#Fig.tight_layout()
plt.suptitle("Osnabrück 2015", fontsize=15)
plt.title("Distribution of Age")
Ax[1].bar([0,1,2,3], [Age0/AllBremen, Age1/AllBremen, Age2/AllBremen, Age3/AllBremen])
Ax[1].set_title("Orbituary Notes")

#Ax[1].set_xticklabels( ['','0-64', '65-74', '75-84', '85+'])
BremenSum = np.sum(SumPerWeekBremen, axis=1) # Achtung: Niedersachsen index richtig?
print('Sum All Bremen 2015:', sum(BremenSum)) # Genau die Zahl, die ich auch Online gefunden habe!

Ax[0].bar([0,1,2,3], BremenSum)
Ax[0].set_title("Official Statistic")
Ax[0].set_xticklabels( ['0-64', '65-74', '75-84', '85+'])

Ax[2].bar([0,1,2,3], [Age0, Age1, Age2, Age3]/BremenSum)
Ax[2].set_title("Fraction covered")
Ax[2].set_xticklabels( ['0-64', '65-74', '75-84', '85+'])
#plt.ylabel("Tote pro Jahr")

plt.show()
Fig.savefig('RegionalCoverage.png')

In [ ]:
## sanity check: portion of region in state
OsnaInhabitants = 273000
OsnaLandInhabitants = 520000
NiedersachsenInhabitants = 7945685
OsnaLandInhabitants / NiedersachsenInhabitants

In [ ]:
import re
df_names = pd.read_csv('/home/cath/Documents/get_rip/src/mortality/Noz_death_data_names.csv', delimiter = ',')
df_names['Merged'] = df_names['BD'].astype(str) + df_names['DD'].astype(str)

print("Namen orig" , len(set(df_names['Merged'])))
df_names.shape
df_names

# only names: 42089
# name + birthdate + deathdate: 44255
# birthdate + deathdate: 43206